## 4.5 튜닝

### 공통 전처리

In [ ]:
# 공통 처리

# 불필요한 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 한글 글꼴 설정
import platform

if platform.system() == 'Windows':
    plt.rc('font', family='Malgun Gothic')
elif platform.system() == 'Darwin':
    plt.rc('font', family='Apple Gothic')

# 데이터프레임 출력용 함수
from IPython.display import display

# 숫자 출력 조정
# 넘파이 부동소수점 출력 자리수 설정
np.set_printoptions(suppress=True, precision=4)

# 판다스 부동소수점 출력 자리수 설정
pd.options.display.float_format = '{:.4f}'.format

# 데이터프레임 모든 필드 출력
pd.set_option("display.max_columns",None)

# 그래프 글꼴 크기 설정
plt.rcParams["font.size"] = 14

# 난수 시드
random_seed = 123

#### 예제 데이터 읽어 들이기

In [ ]:
# 예제 데이터 읽어 들이기
# (유방암 진단 데이터)

# 데이터 읽기
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

# 입력 데이터: x (30차원)
# 정답 데이터: y
x = cancer.data
y = cancer.target

In [ ]:
# 예제 데이터 분할하기

# 데이터 분할 파라미터
test_size = 0.1

# 데이터 분할
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,
    test_size=test_size, random_state=random_seed,
    stratify=y)

# 분할 후 데이터 건수 확인
print(x.shape)
print(x_train.shape)
print(x_test.shape)

### 4.5.1 알고리즘 선택하기

In [ ]:
# 여러 알고리즘 비교하기
# 결과가 같게 random_state 값은 동일하게 둔다

# 선형 회귀
from sklearn.linear_model import LogisticRegression
algorithm1 = LogisticRegression(random_state=random_seed)

# 서포트 벡터 머신(커널)
from sklearn.svm import SVC
algorithm2 = SVC(kernel='rbf', random_state=random_seed)

# 결정 트리
from sklearn.tree import DecisionTreeClassifier
algorithm3 = DecisionTreeClassifier(random_state=random_seed)

# 랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier
algorithm4 = RandomForestClassifier(random_state=random_seed)

# XGBoost
from xgboost import XGBClassifier
algorithm5 = XGBClassifier(random_state=random_seed)

# 알고리즘을 담은 리스트를 생성
algorithms = [algorithm1, algorithm2, algorithm3, algorithm4,
    algorithm5]

In [ ]:
# 여러 알고리즘의 정확도 비교
for algorithm in algorithms:

    # 학습을 수행
    algorithm.fit(x_train, y_train)

    # 정확도 측정
    score = algorithm.score(x_test, y_test)

    # 알고리즘 이름
    name = algorithm.__class__.__name__

    # 정확도, 알고리즘 이름 출력
    print(f'score: {score:.4f}  {name}')

### 4.5.2 하이퍼파라미터 최적화

In [ ]:
# 파라미터 기본값 확인하기
algorithm = SVC(kernel='rbf', random_state=random_seed)
# print(algorithm)
print(algorithm.get_params())

In [ ]:
# gamma의 최적값 구하기
algorithm = SVC(kernel='rbf', random_state=random_seed)
gammas = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]

for gamma in gammas:
    algorithm.gamma = gamma
    algorithm.fit(x_train, y_train)
    score = algorithm.score(x_test, y_test)
    print(f'score: {score:.4f}  gamma: {gamma}')

In [ ]:
# C의 최적값 구하기
# gamma는 앞서 구한 최적값 0.001을 적용

Cs = [1,  10,  100, 1000, 10000]
for C in Cs:
    algorithm = SVC(kernel='rbf',
        gamma=0.001, C=C,
        random_state=random_seed)
    algorithm.fit(x_train, y_train)
    score = algorithm.score(x_test, y_test)
    print(f'score: {score:.4f}  C: {C}')

### 4.5.3 교차 검증법

In [ ]:
# 특정 알고리즘에 교차 검증법 적용하기

# 알고리즘 정의
algorithm = SVC(kernel='rbf',random_state=random_seed,
    gamma=0.001, C=1)

# 데이터 분할 시 분포가 치우치지 않도록 StratifiedKFold를 사용함
from sklearn.model_selection import StratifiedKFold
stratifiedkfold = StratifiedKFold(n_splits=3)

# 교차 검증법 수행 (분할 수=3)
from sklearn.model_selection import cross_val_score
scores = cross_val_score(algorithm , x_train, y_train,
    cv=stratifiedkfold)

# 평균을 계산
mean = scores.mean()

# 결과 확인
print(f'평균 정확도: {mean:.4f}  개별 정확도: {scores}')

In [ ]:
# 후보 알고리즘 리스트 만들기

from sklearn.linear_model import LogisticRegression
algorithm1 = LogisticRegression(random_state=random_seed)

from sklearn.svm import SVC
algorithm2 = SVC(kernel='rbf',random_state=random_seed,
    gamma=0.001, C=1)

from sklearn.tree import DecisionTreeClassifier
algorithm3 = DecisionTreeClassifier(random_state=random_seed)

from sklearn.ensemble import RandomForestClassifier
algorithm4 = RandomForestClassifier(random_state=random_seed)

from xgboost import XGBClassifier
algorithm5 = XGBClassifier(random_state=random_seed)

algorithms = [algorithm1, algorithm2, algorithm3, algorithm4,
    algorithm5]

In [ ]:
# 여러 알고리즘의 정확도 비교하기

# 데이터 분할 시 분포가 치우치지 않도록 StratifiedKFold를 사용함
from sklearn.model_selection import StratifiedKFold
stratifiedkfold = StratifiedKFold(n_splits=3)

from sklearn.model_selection import cross_val_score
for algorithm in algorithms:
    # 교차 검증법 수행
    scores = cross_val_score(algorithm , x_train, y_train,
        cv=stratifiedkfold)
    score = scores.mean()
    name = algorithm.__class__.__name__
    print(f'평균 정확도: {score:.4f}  개별 정확도: {scores}  {name}')

### 4.5.4 그리드 서치

In [ ]:
# 그리드 서치와 교차 검증법을 모두 적용한 최적 파라미터 탐색
params = {
      'C':[1, 10, 100, 1000, 10000],
      'gamma':[1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
}
algorithm = SVC(random_state=random_seed)

from sklearn.model_selection import StratifiedKFold
stratifiedkfold = StratifiedKFold(n_splits=3)

from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(algorithm, params, cv=stratifiedkfold)
gs.fit(x_train, y_train)

# 가장 뛰어난 모델로 검증 데이터를 분류
best = gs.best_estimator_
best_pred = best.predict(x_test)
print(best.get_params())

In [ ]:
# 정확도 계산
score = best.score(x_test, y_test)
print(f'정확도: {score:.4f}')

# 혼동행렬 출력하기
from sklearn.metrics import confusion_matrix
print()
print('혼동행렬')
print(confusion_matrix(y_test, best_pred))